In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from env import get_db_url

import warnings
warnings.filterwarnings("ignore")


Go through the lesson commenting code, adding docstrings, and adding markdown to support what is happening.

Bonus:

Discover users who are accessing our curriculum pages way beyond the end of their codeup time. What would the dataframe look like? Use time series method for detecting anomalies, like exponential moving average with %b.

Can you label students who are viewing both the web dev and data science curriculum? Can you label students by the program they are in? Can you label users by student vs. staff?

In [13]:
query = """SELECT * FROM logs LEFT JOIN cohorts ON logs.user_id = cohorts.id"""
df = pd.read_sql(query, get_db_url('curriculum_logs'))

In [16]:
df.describe()

,user_id,cohort_id,id,program_id
count,900223.000000,847330.000000,73739.000000,73739.000000
mean,458.825707,48.501049,30.566213,1.561670
std,249.296767,32.795482,35.540653,0.659918
min,1.000000,1.000000,1.000000,1.000000
25%,269.000000,28.000000,11.000000,1.000000
50%,475.000000,33.000000,12.000000,1.000000
75%,660.000000,57.000000,53.000000,2.000000
max,981.000000,139.000000,139.000000,4.000000


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900223 entries, 0 to 900222
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   date        900223 non-null  object 
 1   time        900223 non-null  object 
 2   path        900222 non-null  object 
 3   user_id     900223 non-null  int64  
 4   cohort_id   847330 non-null  float64
 5   ip          900223 non-null  object 
 6   id          73739 non-null   float64
 7   name        73739 non-null   object 
 8   slack       73739 non-null   object 
 9   start_date  73739 non-null   object 
 10  end_date    73739 non-null   object 
 11  created_at  73739 non-null   object 
 12  updated_at  73739 non-null   object 
 13  deleted_at  0 non-null       object 
 14  program_id  73739 non-null   float64
dtypes: float64(3), int64(1), object(11)
memory usage: 103.0+ MB


In [15]:
df.isna().sum()

date               0
time               0
path               1
user_id            0
cohort_id      52893
ip                 0
id            826484
name          826484
slack         826484
start_date    826484
end_date      826484
created_at    826484
updated_at    826484
deleted_at    900223
program_id    826484
dtype: int64